In [8]:
# Allow import of modules from src/
import sys
sys.path.append("..")

In [13]:
from pathlib import Path

import logging

import yappi

from pandas import DataFrame

from muCh import muCh
from checking import Checker
from parsing import Parser

In [14]:
systems_dir = Path("../../material/dining/")
formulas_dir = Path("../../material/dining/")

results_path = Path("../../profiling/dining/2022-12-13_10-51.csv")

logging.basicConfig(format='%(asctime)s:%(levelname)s:%(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO)

In [3]:
algorithm = Checker.Algorithm.NAIVE

system_paths = [file for file in systems_dir.iterdir() if file.suffix == '.aut']
formula_paths = [file for file in formulas_dir.iterdir() if file.suffix == '.mcf']

In [4]:
formulas = {}
for formula_path in formula_paths:
    logging.info("Parsing formula '%s'." % formula_path.name)
    formulas[formula_path.name] = Parser.parse_formula(formula_path)

systems = {}
for system_path in system_paths:
    logging.info("Parsing system '%s'." % system_path.name)
    systems[system_path.name] = Parser.parse_system(system_path)

20:23:16:INFO:Parsing formula 'invariantly_inevitably_eat.mcf'.
20:23:16:INFO:Parsing formula 'invariantly_plato_starves.mcf'.
20:23:16:INFO:Parsing formula 'invariantly_possibly_eat.mcf'.
20:23:16:INFO:Parsing formula 'plato_infinitely_often_can_eat.mcf'.
20:23:16:INFO:Parsing system 'dining_10.aut'.
20:25:18:INFO:Parsing system 'dining_2.aut'.
20:25:18:INFO:Parsing system 'dining_3.aut'.
20:25:19:INFO:Parsing system 'dining_4.aut'.
20:25:19:INFO:Parsing system 'dining_5.aut'.
20:25:19:INFO:Parsing system 'dining_6.aut'.
20:25:20:INFO:Parsing system 'dining_7.aut'.
20:25:23:INFO:Parsing system 'dining_8.aut'.
20:25:40:INFO:Parsing system 'dining_9.aut'.


In [5]:
statistics = []

for system_name, system in sorted(systems.items(), key=lambda x: int(x[0].split("_")[1].split(".")[0])):
    for formula_name, formula in formulas.items():
        for algorithm in Checker.Algorithm:
            logging.info("Checking system %s on formula %s with algorithm %s." % (system_name, formula_name, algorithm.name))

            with yappi.run():
                satisfied, _, ifp = Checker.solver(system, formula, algorithm)

            ttot = yappi.get_func_stats(filter_callback=lambda x: yappi.func_matches(x, [Checker.solver]))[0].ttot

            statistics.append((system_name, formula_name, algorithm.name, satisfied, ifp, ttot))

20:26:36:INFO:Checking system dining_2.aut on formula invariantly_inevitably_eat.mcf with algorithm NAIVE.
20:26:36:INFO:Checking system dining_2.aut on formula invariantly_inevitably_eat.mcf with algorithm EMERSON_LEI.
20:26:36:INFO:Checking system dining_2.aut on formula invariantly_plato_starves.mcf with algorithm NAIVE.
20:26:36:INFO:Checking system dining_2.aut on formula invariantly_plato_starves.mcf with algorithm EMERSON_LEI.
20:26:36:INFO:Checking system dining_2.aut on formula invariantly_possibly_eat.mcf with algorithm NAIVE.
20:26:36:INFO:Checking system dining_2.aut on formula invariantly_possibly_eat.mcf with algorithm EMERSON_LEI.
20:26:36:INFO:Checking system dining_2.aut on formula plato_infinitely_often_can_eat.mcf with algorithm NAIVE.
20:26:36:INFO:Checking system dining_2.aut on formula plato_infinitely_often_can_eat.mcf with algorithm EMERSON_LEI.
20:26:36:INFO:Checking system dining_3.aut on formula invariantly_inevitably_eat.mcf with algorithm NAIVE.
20:26:36:IN

In [6]:
df = DataFrame(statistics, columns = ["System", "Formula", "Algorithm", "satisfied", "ifp", "ttot"])

In [17]:
# Print profiling results
df

In [18]:
# Save data frame
df.to_csv(results_path)